In [1]:
import pandas as pd
import numpy as np

In [39]:
years = np.arange(2015,2019,1)
models = ['bot', 'top']

def getNDCG(k, preds, y):
    ndcg = 0
    for j in range(0, k):
        if preds[j] == y:
            ndcg = 1 / np.log2(j + 2)
    return ndcg

topk = 10
for model in models:
    for year in years:
        data = pd.read_csv(str(year)+model+'15new.csv', header=0, sep=',')
        data['rank_latest'] = data.groupby(['userId'])['timestamp'].rank(method='first', ascending=False)
        train_ids, test_ids = data.index.values[data['rank_latest'] > 1], data.index.values[data['rank_latest'] == 1]
        train, test = data.iloc[train_ids,:].reset_index(drop=True),data.iloc[test_ids,:].reset_index(drop=True)
        assert len(data) == len(train)+len(test)
        most_pop = data.groupby(by='movieId').size().reset_index(name='pop')
        most_pop_items = most_pop.sort_values(by='pop',ascending=False)['movieId'].values
        raw_rated = dict(train.groupby(by='userId')['movieId'].unique())
        rated = {}
        for user in raw_rated:
            rated[user] = set(raw_rated[user])
        
        hit = 0
        ndcg = 0
        for user, item in dict(zip(test['userId'],test['movieId'])).items():
            index = 0
            recommended = []
            while len(recommended)!=topk:
                if most_pop_items[index] not in rated[user]:
                    recommended.append(most_pop_items[index])
                index+=1
            if item in recommended:
                hit+=1
                ndcg+=getNDCG(topk,recommended,item)
        
        print(str(year)+model, hit/len(test), ndcg/len(test))
        

2015bot 0.041638456330399455 0.019834863472566185
2016bot 0.04061895551257253 0.019553836721070944
2017bot 0.03484654731457801 0.019465210671244095
2018bot 0.03543107811709128 0.01572130243859439
2015top 0.1525050778605281 0.07372256104789522
2016top 0.15503645290879334 0.07415032211145366
2017top 0.1504156010230179 0.07049173922114242
2018top 0.14847308925257297 0.07634970702221117


In [63]:
test_ids

array([   14,    29,    44, ..., 88874, 88889, 88904], dtype=int64)

In [40]:
data = pd.read_csv(str(year)+'top15new.csv', header=0, sep=',')
data1 = pd.read_csv(str(year)+'bot15new.csv', header=0, sep=',')
        

In [46]:
newdata = pd.concat([data,data1]).reset_index(drop=True)

In [51]:
noduplicate = newdata.drop_duplicates()

In [55]:
noduplicate.groupby(by='userId').size().max()

30

In [56]:
data['userId'].unique()

array([    75,    108,    195, ..., 162394, 162419, 162424], dtype=int64)

In [57]:
alldata =pd.read_csv(str(year)+'all.csv', header=0, sep=',')

In [61]:
(alldata['userId'].unique() == data['userId'].unique()).sum()

5927

In [62]:
alldata['userId'].nunique()

5927

In [38]:
recommended

[3793, 3793, 3793, 3793, 3793, 3793, 3793, 3793, 3793, 3793]

In [ ]:
2015bot 0.03876100203114421 0.018071898402933138
2016bot 0.03868471953578337 0.018553030244139068
2017bot 0.03308823529411765 0.01855568645314957
2018bot 0.03374388392103931 0.014629896941296608
2015top 0.08937034529451592 0.03588820270456202
2016top 0.08436244606457373 0.034563421482119355
2017top 0.08967391304347826 0.03579690594228111
2018top 0.10578707609245824 0.0433313412973615

In [34]:
dict(zip(test['userId'],test['movieId'])).items()

dict_items([(75, 50), (108, 104879), (195, 79132), (254, 1682), (264, 2959), (275, 8368), (321, 177593), (355, 1270), (390, 104841), (403, 356), (419, 3556), (475, 122900), (486, 112852), (491, 58559), (498, 168366), (541, 122912), (577, 58559), (607, 7153), (615, 47099), (675, 48780), (698, 101962), (739, 58559), (749, 68157), (755, 8874), (763, 164179), (800, 168250), (819, 7438), (845, 122282), (865, 134855), (910, 122904), (935, 318), (967, 59315), (1011, 87234), (1034, 112852), (1060, 58559), (1187, 48394), (1194, 47), (1223, 190939), (1226, 68157), (1238, 1198), (1241, 7361), (1265, 7153), (1272, 356), (1355, 2571), (1358, 2028), (1386, 4306), (1390, 539), (1428, 1196), (1512, 4973), (1520, 115713), (1524, 49272), (1533, 1196), (1585, 3793), (1628, 60069), (1673, 104881), (1682, 2959), (1711, 8368), (1779, 253), (1815, 1036), (1816, 2571), (1820, 2028), (1822, 2959), (1963, 356), (1974, 57669), (1987, 2502), (2006, 4993), (2045, 1198), (2077, 8961), (2084, 53121), (2121, 115617),

In [30]:
most_pop.sort_values(by='pop',ascending=False)

,movieId,pop
98,318,3124
752,2571,2849
1557,7153,2117
2685,79132,2032
851,2959,1945
...,...,...
2523,69995,1
2524,70015,1
621,2120,1
620,2119,1
